<a href="https://colab.research.google.com/github/Malikumair786/Machine_Learning/blob/main/Project_4_Fake_News_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Workflow:**

News Data --> data preprocessing --> train_test_split --> Logistic Regression Model --> Trained Logistic Regression model --> feed New Data --> predict the news is real or Fake

About the dataset:
1. id: Unique id for a news article
2. title: the title of a news article
3. author: Author of the news article
4. text: The text of the article; could be incomplete
5. label: A label that marks wheather the news article is real or fake.

1: Fake News

0: Real News

importing the dependencies

In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
#printing the stopwords in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

Data Preprocessing

In [6]:
#loading the data set to a pandas dataframe
news_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/train.csv')


In [7]:
#number of rows and columns
news_dataset.shape

(20800, 5)

In [8]:
#print the first five rows of the dataFrame
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
# coutning the number of missing alues in the dataset
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [10]:
# replacing the null values with empty string
news_dataset = news_dataset.fillna('')

In [12]:
#merging the author name and news title
news_dataset['content'] = news_dataset['author']+' '+news_dataset['title']

In [13]:
print(news_dataset['content'])

0        Darrell Lucus House Dem Aide: We Didn’t Even S...
1        Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2        Consortiumnews.com Why the Truth Might Get You...
3        Jessica Purkiss 15 Civilians Killed In Single ...
4        Howard Portnoy Iranian woman jailed for fictio...
                               ...                        
20795    Jerome Hudson Rapper T.I.: Trump a ’Poster Chi...
20796    Benjamin Hoffman N.F.L. Playoffs: Schedule, Ma...
20797    Michael J. de la Merced and Rachel Abrams Macy...
20798    Alex Ansary NATO, Russia To Hold Parallel Exer...
20799              David Swanson What Keeps the F-35 Alive
Name: content, Length: 20800, dtype: object


Now we will be using the content column data to make predictions

In [14]:
#Seperating the data and labels
X = news_dataset.drop(['label'], axis = 1)
Y = news_dataset['label']

In [15]:
print(X)
print(Y)

          id                                              title  \
0          0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1          1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2          2                  Why the Truth Might Get You Fired   
3          3  15 Civilians Killed In Single US Airstrike Hav...   
4          4  Iranian woman jailed for fictional unpublished...   
...      ...                                                ...   
20795  20795  Rapper T.I.: Trump a ’Poster Child For White S...   
20796  20796  N.F.L. Playoffs: Schedule, Matchups and Odds -...   
20797  20797  Macy’s Is Said to Receive Takeover Approach by...   
20798  20798  NATO, Russia To Hold Parallel Exercises In Bal...   
20799  20799                          What Keeps the F-35 Alive   

                                          author  \
0                                  Darrell Lucus   
1                                Daniel J. Flynn   
2                             Consortiu

Stemming: 

Stemming is the process of reducing a word to its root word

Example: 
actor, actress, acting --> root word is act

In [16]:
port_stem = PorterStemmer()

In [17]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z],',' ',content)    # this line removes everything that is not alphabet
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()   # conerting text to list
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)
  return stemmed_content

In [18]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [19]:
print(news_dataset['content'])

0        darrel lucu hous dem aide: didn’t even see com...
1        daniel j. flynn flynn: hillari clinton, big wo...
2                  consortiumnews.com truth might get fire
3        jessica purkiss 15 civilian kill singl us airs...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper t.i.: trump ’poster child ...
20796    benjamin hoffman n.f.l. playoffs: schedule, ma...
20797    michael j. de la merc rachel abram macy’ said ...
20798    alex ansari nato, russia hold parallel exercis...
20799                         david swanson keep f-35 aliv
Name: content, Length: 20800, dtype: object


In [20]:
#seperating the data and label
X = news_dataset['content'].values
Y = news_dataset['label'].values

In [21]:
print(X)

['darrel lucu hous dem aide: didn’t even see comey’ letter jason chaffetz tweet'
 'daniel j. flynn flynn: hillari clinton, big woman campu - breitbart'
 'consortiumnews.com truth might get fire' ...
 'michael j. de la merc rachel abram macy’ said receiv takeov approach hudson’ bay - new york time'
 'alex ansari nato, russia hold parallel exercis balkan'
 'david swanson keep f-35 aliv']


In [22]:
print(Y)

[1 0 1 ... 0 1 1]


In [23]:
Y.shape

(20800,)

In [24]:
X.shape

(20800,)

In [26]:
# Now converting textual data to numerical data
vectorizer = TfidfVectorizer() 
vectorizer.fit(X)

X = vectorizer.transform(X)

In [27]:
print(X)

  (0, 19084)	0.27315635150958634
  (0, 16460)	0.23676064517956458
  (0, 11060)	0.33384522056560495
  (0, 10735)	0.2682220926318631
  (0, 9681)	0.22757176689298136
  (0, 8821)	0.205341824533184
  (0, 6421)	0.2142258791026174
  (0, 5244)	0.27468869329117757
  (0, 4983)	0.2512923264945339
  (0, 4751)	0.3304457115379666
  (0, 3940)	0.22664699692052692
  (0, 3391)	0.3375689613898566
  (0, 797)	0.3646500188253279
  (1, 20403)	0.299521873400654
  (1, 8597)	0.19810516182474192
  (1, 7089)	0.7114899247272568
  (1, 4716)	0.26268912792483357
  (1, 3766)	0.19062864013183065
  (1, 3088)	0.38708204516205785
  (1, 2701)	0.15460262441734718
  (1, 2246)	0.29282032322324314
  (2, 19002)	0.41491113753784553
  (2, 11864)	0.49151393723208897
  (2, 7639)	0.34605253138342823
  (2, 6956)	0.39293503470255664
  :	:
  (20797, 15129)	0.2718547014681534
  (20797, 14893)	0.24707379158386517
  (20797, 12671)	0.08013411384942215
  (20797, 11830)	0.17405416749037161
  (20797, 11754)	0.2945753459972408
  (20797, 11167)

Splitting the dataset to training and test data

In [28]:
X_train , X_test, Y_train , Y_test = train_test_split(X,Y,test_size = 0.2,stratify = Y,random_state = 2)

In [29]:
print(X.shape,X_train.shape,X_test.shape)

(20800, 21813) (16640, 21813) (4160, 21813)


Training the model: logistic regression

In [30]:
model = LogisticRegression()

In [31]:
model.fit(X_train,Y_train)

LogisticRegression()

Evaluation

Accuracy Score

In [32]:
#Accuracy score on the training data prediction
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction,Y_train)

In [33]:
print("The Accuracy Score of the training data: ",training_data_accuracy)

The Accuracy Score of the training data:  0.98671875


In [34]:
#Accuracy score on the test data prediction
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction,Y_test)

In [35]:
print("The Accuracy Score of the test data: ",test_data_accuracy)

The Accuracy Score of the test data:  0.9776442307692308


Making a predictive System

In [36]:
X_new = X_test[0]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):      #0--> real news
  print("The news is Real")
else:
  print("The news is fake")

[1]
The news is fake


In [37]:
print(Y_test[0])

1


In [38]:
X_new = X_test[1]

prediction = model.predict(X_new)
print(prediction)

if(prediction[0] == 0):      #0--> real news
  print("The news is Real")
else:
  print("The news is fake")

[0]
The news is Real


In [39]:
print(Y_test[1])

0
